# テキスト分類

* DistilBERTをFine-tuningしてTwitterデータの感情検出器を作る
* angry, love, fear, joy, sadness, surpriseの6感情

## データセット

* [emotionsデータセット](https://huggingface.co/datasets/emotion)

```
Emotion は、怒り、恐れ、喜び、愛、悲しみ、驚きの 6 つの基本的な感情を含む英語の Twitter メッセージのデータセットです。詳細については、論文を参照してください
```

### Hugging Face Datasets

* `list_datasets()` でデータセット一覧を確認できる
* `load_dataset()` でダウンロードできる

In [ ]:
from datasets import list_datasets

all_datasets = list_datasets()
print(f"There are {len(all_datasets)} datasets currently available on the Hub")
print(f"The first 10 are: {all_datasets[:10]}")

In [ ]:
from datasets import load_dataset

emotions = load_dataset("emotion")

In [ ]:
emotions

In [ ]:
train_ds = emotions["train"]
train_ds

In [ ]:
len(train_ds)

In [ ]:
train_ds[0]

In [ ]:
train_ds.column_names

In [ ]:
train_ds.features

In [ ]:
train_ds[:5]

In [ ]:
train_ds["text"][:5]

### DataFrameへの変換

* `set_format()` でDatasetの出力形式を変更できる
* `int2str()` でラベルIDを文字列に変換できる

In [ ]:
import pandas as pd

emotions.set_format(type="pandas")

# 出力がpandasのDataFrameになった
emotions["train"][0]

In [ ]:
# trainの全データを取得
df = emotions["train"][:]
df.head()

In [ ]:
emotions["train"].features["label"]

In [ ]:
emotions["train"].features["label"].int2str([0, 2])

In [ ]:
def label_int2str(row):
    return emotions["train"].features["label"].int2str(row)

df["label_name"] = df["label"].apply(label_int2str)
df.head()

### クラス分布の確認

In [ ]:
import matplotlib.pyplot as plt

df["label_name"].value_counts(ascending=True).plot.barh()
plt.title("Frequency of Classes")
plt.show()

### ツイートの長さはどれくらい？

* Transformerモデルには最大コンテキストサイズという入力系列長の制限がある
* DistilBERTは512トークン

In [ ]:
df["text"].str.split()[0]

In [ ]:
df["Words Per Tweet"] = df["text"].str.split().apply(len)
df.boxplot("Words Per Tweet", by="label_name", grid=False, showfliers=False, color="black")
plt.suptitle("")
plt.xlabel("")
plt.show()

In [ ]:
# データセットの出力形式を戻す
emotions.reset_format()

## テキストからトークンへ

* 通常、最適な単語分割はコーパスから学習する
* もっともシンプルなやり方は文字トークン化と単語トークン化
* この2つのやり方もデータによって変わるので学習と言ってもよさそう

### 文字トークン化

In [ ]:
text = "Tokenizing text is a core task of NLP."
tokenized_text = list(text)
print(tokenized_text)

In [ ]:
print(sorted(set(tokenized_text)))

In [ ]:
token2idx = {ch: idx for idx, ch in enumerate(sorted(set(tokenized_text)))}
print(token2idx)

In [ ]:
len(token2idx)

In [ ]:
input_ids = [token2idx[token] for token in tokenized_text]
print(input_ids)

In [ ]:
# one-hotベクトルへの変換
import torch
import torch.nn.functional as F

input_ids = torch.tensor(input_ids)
one_hot_encodings = F.one_hot(input_ids, num_classes=len(token2idx))
one_hot_encodings.shape

### サブワードトークン化

* 文字トークン化と単語トークン化の中間
* コーパスからトークン化を学習する
* 頻出単語は単語として使う、そうでないものはより小さな単位に分割する
* WordPiece: BERTとDistilBERTのTokenizer
* `AutoTokenizer.from_pretrained()` を使うと指定したモデルのTokenizerをロードできる
* [Autoがついている場合](https://huggingface.co/docs/transformers/model_doc/auto)はモデル名から自動判定する
* モデルに対応するTokenizerを使う必要がある

In [ ]:
from transformers import AutoTokenizer

model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
tokenizer

In [ ]:
# Autoクラスを使わずにモデルごとのクラスも使える
from transformers import DistilBertTokenizer

distilbert_tokenizer = DistilBertTokenizer.from_pretrained(model_ckpt)
distilbert_tokenizer

* Tokenizerをメソッドとして使うとinput_idsに変換できる
* `convert_ids_to_tokens` を使うとinput_idsをトークンに戻せる

In [ ]:
text = "Tokenizing text is a core task of NLP."
encoded_text = tokenizer(text)
print(encoded_text)

* [CLS] と [SEP] のような特別なトークンが付与される（モデルによって異なる）
* `##` は分割されたトークン、文字列に変換するときは前のトークンとマージされる
* 一般的でない単語は `##` で分割されやすい

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(tokens)

* `convert_to_tokens_to_string()` でトークン列を文字列に変換できる

In [ ]:
print(tokenizer.convert_tokens_to_string(tokens))

In [ ]:
# ボキャブラリーサイズ
tokenizer.vocab_size

In [ ]:
# 最大コンテキストサイズ
tokenizer.model_max_length

In [ ]:
# モデルのforwardパスで期待するフィールド名
tokenizer.model_input_names

### コーパス全体のトークン化

* DatasetDictのmapを使うと全データをまとめてトークン化できる
* attention_maskはパディングされた部分が0になる

In [ ]:
def tokenize(batch):
    # input_idsとattention_maskのfeatureが追加される
    return tokenizer(batch["text"], padding=True, truncation=True)

In [ ]:
batch = emotions["train"][:2]
batch

In [ ]:
print(tokenize(batch))

In [ ]:
# batch_size=Noneを指定すると全データを1つのバッチとみなす
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)

In [ ]:
emotions_encoded["train"]

In [ ]:
# 全データで最長の系列をもとにpaddingされるので無駄が多い
print(emotions_encoded["train"][0])

## テキスト分類器の学習

1. 特徴抽出器として利用
2. ファインチューニング

### 特徴抽出器としてのTransformer

In [ ]:
from transformers import AutoModel

model_ckpt = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)

In [ ]:
text = "this is a test"
inputs = tokenizer(text, return_tensors="pt")
inputs

In [ ]:
inputs['input_ids'].size()

In [ ]:
inputs = {k: v.to(device) for k, v in inputs.items()}
inputs

In [ ]:
with torch.no_grad():
    outputs = model(**inputs)
outputs

In [ ]:
# 最後の隠れ状態の出力
# 入力の6個のトークンそれぞれに対して768次元のベクトルが出力される
# [batch_size, n_tokens, hidden_dim]
outputs.last_hidden_state.size()

In [ ]:
# 分類では [CLS] トークン（系列の最初）に出てくる隠れ状態だけを使うのが一般的
outputs.last_hidden_state[:, 0].size()

In [ ]:
# データ全体に対して特徴量を抽出する
def extract_hidden_states(batch):
    inputs = {k: v.to(device) for k, v in batch.items() if k in tokenizer.model_input_names}
    # 最後の隠れ状態を抽出
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state
    # [CSL] トークンに対するベクトルを返す
    # hidden_stateのfeatureが追加される
    return {"hidden_state": last_hidden_state[:, 0].cpu().numpy()}

In [ ]:
# 出力をnumpyではなくテンソルにするように変更
emotions_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
emotions_encoded["train"][0]

In [ ]:
# デフォルトでbatch_size=1000が使われる
emotions_hidden = emotions_encoded.map(extract_hidden_states, batched=True)
emotions_hidden

In [ ]:
emotions_hidden["train"].column_names

In [ ]:
emotions_hidden["train"][0]['hidden_state'].size()

In [ ]:
# 特徴行列の作成
import numpy as np

X_train = np.array(emotions_hidden["train"]["hidden_state"])
X_valid = np.array(emotions_hidden["validation"]["hidden_state"])
y_train = np.array(emotions_hidden["train"]["label"])
y_valid = np.array(emotions_hidden["validation"]["label"])
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

In [ ]:
# 学習データセットの可視化
# UMAPは特徴が[0, 1]区間になるようにスケールするとうまく機能する
from umap import UMAP
from sklearn.preprocessing import MinMaxScaler

# 特徴量を[0, 1]区間にスケールする
X_scaled = MinMaxScaler().fit_transform(X_train)
X_scaled.shape

In [ ]:
# UMAPの初期化とfit
mapper = UMAP(n_components=2, metric="cosine").fit(X_scaled)
mapper

In [ ]:
mapper.embedding_

In [ ]:
# 2次元埋め込みのDataFrameを作成
df_emb = pd.DataFrame(mapper.embedding_, columns=["X", "Y"])
df_emb["label"] = y_train
df_emb.head()

In [ ]:
# 圧縮されたデータを可視化
# 元のDistilBERTは感情分類をするために訓練されたモデルでないので分類できなくて当然
# 言語モデルとして訓練されていても多少は感情の違いを考慮できていることがわかる
fig, axes = plt.subplots(2, 3, figsize=(7, 5))
axes = axes.flatten()
cmaps = ["Greys", "Blues", "Oranges", "Reds", "Purples", "Greens"]
labels = emotions["train"].features["label"].names

for i, (label, cmap) in enumerate(zip(labels, cmaps)):
    df_emb_sub = df_emb.query(f"label == {i}")
    axes[i].hexbin(df_emb_sub["X"], df_emb_sub["Y"], cmap=cmap,
                   gridsize=20, linewidths=(0,))
    axes[i].set_title(label)
    axes[i].set_xticks([]), axes[i].set_yticks([])

plt.tight_layout()
plt.show()

In [ ]:
# 単純な分類器（ロジスティック回帰）の学習
# DistilBERTの特徴量からラベルを予測するモデルを訓練する
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(max_iter=3000)
lr_clf.fit(X_train, y_train)
lr_clf.score(X_valid, y_valid)

In [ ]:
# ベースラインとなるダミー分類器
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)
dummy_clf.score(X_valid, y_valid)

In [ ]:
# 混合行列を可視化
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds, normalize="true")
    fig, ax = plt.subplots(figsize=(6, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
    plt.title("Normalized confusion matrix")
    plt.show()

In [ ]:
y_preds = lr_clf.predict(X_valid)
plot_confusion_matrix(y_preds, y_valid, labels)

### TransformerのFine-tuning

* 隠れ状態を固定的な特徴量として使わない
* 特徴抽出も訓練対象とする
* ヘッダが微分可能である必要があるのでLogistic Regressionではなくニューラルネットを使う
* Trainer API

In [ ]:
# 事前学習済みモデルのロード
from transformers import AutoModelForSequenceClassification

model_ckpt = "distilbert-base-uncased"
num_labels = 6
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels)

In [ ]:
# 性能指標の定義
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    # 正解ラベル
    labels = pred.label_ids
    # 予測ラベル
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:
# 学習パラメータを定義
from transformers import Trainer, TrainingArguments

batch_size = 64
logging_steps = len(emotions_encoded["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-emotion"  # これがHubに登録される名前になる
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=True, 
                                  log_level="error")

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
# Trainerで訓練（Fine-tuning）
from transformers import Trainer

trainer = Trainer(model=model,
                  args=training_args, 
                  compute_metrics=compute_metrics,
                  train_dataset=emotions_encoded["train"],
                  eval_dataset=emotions_encoded["validation"],
                  tokenizer=tokenizer)
trainer.train()

In [ ]:
# 検証データに対する予測
preds_output = trainer.predict(emotions_encoded["validation"])

In [ ]:
preds_output

In [ ]:
preds_output.metrics

In [ ]:
preds_output.label_ids

In [ ]:
preds_output.predictions.shape

In [ ]:
y_preds = np.argmax(preds_output.predictions, axis=1)
y_preds.shape

In [ ]:
plot_confusion_matrix(y_preds, y_valid, labels)

In [ ]:
# エラー分析
from torch.nn.functional import cross_entropy

def forward_pass_with_label(batch):
    # Place all input tensors on the same device as the model
    inputs = {k:v.to(device) for k,v in batch.items() if k in tokenizer.model_input_names}

    with torch.no_grad():
        output = model(**inputs)
        pred_label = torch.argmax(output.logits, axis=-1)
        loss = cross_entropy(output.logits, batch["label"].to(device), reduction="none")

    # Place outputs on CPU for compatibility with other dataset columns   
    return {"loss": loss.cpu().numpy(), 
            "predicted_label": pred_label.cpu().numpy()}

In [ ]:
emotions_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
# 損失の値を計算
emotions_encoded["validation"] = emotions_encoded["validation"].map(forward_pass_with_label,
                                                                    batched=True,
                                                                    batch_size=16)

In [ ]:
emotions_encoded

In [ ]:
emotions_encoded.set_format("pandas")

In [ ]:
cols = ["text", "label", "predicted_label", "loss"]
df_test = emotions_encoded["validation"][:][cols]
df_test["label"] = df_test["label"].apply(label_int2str)
df_test["predicted_label"] = (df_test["predicted_label"].apply(label_int2str))

In [ ]:
df_test.head()

In [ ]:
# lossが大きい（正解との乖離が激しい）データ
df_test.sort_values("loss", ascending=False).head(10)

In [ ]:
# lossが小さい（正解と一致）データ
df_test.sort_values("loss", ascending=True).head(10)

In [ ]:
# モデルの保存と共有
trainer.push_to_hub(commit_message="Training completed!")

In [ ]:
# パイプラインを使って推論
from transformers import pipeline

model_id = "f2forest/distilbert-base-uncased-finetuned-emotion"
classifier = pipeline("text-classification", model=model_id)

In [ ]:
custom_tweet = "I saw a movie today and it was really good."
preds = classifier(custom_tweet, return_all_scores=True)
preds

In [ ]:
preds_df = pd.DataFrame(preds[0])
plt.bar(labels, 100 * preds_df["score"], color='C0')
plt.title(f'"{custom_tweet}"')
plt.ylabel("Class probability (%)")
plt.show()